In [149]:
import pandas as pd
import json

from serlo.mysql import db

In [150]:
# To look for other entity types e.g. ExerciseGroups (id=4) change type_id.
grouped_exercises = pd.read_sql("""
    SELECT entity.id
    FROM entity
    JOIN uuid on entity.id = uuid.id
    WHERE type_id = 5
        AND uuid.trashed = 0;
""", db)
grouped_exercises.fillna(0, inplace=True)
grouped_exercises.head()
        

/tmp/ipykernel_82243/1036390929.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  grouped_exercises = pd.read_sql("""


,id
0,2219
1,2225
2,2229
3,2233
4,2239


In [151]:
entities = pd.read_sql("""
    select entity.id, entity_revision_field.value
    from entity
    join uuid on uuid.id = entity.id
    join entity_revision_field on entity_revision_field.entity_revision_id = entity.current_revision_id
    where entity_revision_field.field = "content" and uuid.trashed = 0
""", db)
entities.fillna(0, inplace=True)

/tmp/ipykernel_82243/719409883.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  entities = pd.read_sql("""


In [152]:
def list_injections(content, inj_list):
    if isinstance(content, dict):
        if "plugin" in content and content["plugin"] == "injection":
            inj_list.append(content["state"])
        else:
            for child in content.values():
                list_injections(child, inj_list)
    elif isinstance(content, list):
        for child in content:
            list_injections(child, inj_list)
    


In [153]:
entity_inj_dict = dict()
for row in entities.iterrows():
    entity_id = row[1].id
    try:
        entity = json.loads(row[1].value)
        inj_list = []
        list_injections(entity, inj_list)
        entity_inj_dict[entity_id] =  inj_list
    except:
        print (row)


(3711, id                                             18865
value    https://www.youtube.com/watch?v=cIzM2XBduuY
Name: 3711, dtype: object)
(3992, id                                            24037
value    http://www.youtube.com/watch?v=90CP8QPV-uc
Name: 3992, dtype: object)
(3993, id                                            24039
value    http://www.youtube.com/watch?v=9NTbBn2slJc
Name: 3993, dtype: object)
(3994, id                                            24041
value    http://www.youtube.com/watch?v=R1bZazTLRK4
Name: 3994, dtype: object)
(3995, id                                            24043
value    http://www.youtube.com/watch?v=B-vaREvWcc0
Name: 3995, dtype: object)
(3996, id                                            24048
value    http://www.youtube.com/watch?v=OBA2lHCh4qw
Name: 3996, dtype: object)
(3997, id                                            24050
value    http://www.youtube.com/watch?v=Nt-EiXjuh_o
Name: 3997, dtype: object)
(3998, id                       

In [155]:
grouped_ex_injections = []
for key in entity_inj_dict:
    for id in entity_inj_dict[key]:
        
        grouped_ex_injections.append({"id":key, "injection":id})
df_inj = pd.DataFrame(grouped_ex_injections)
df_inj = df_inj[df_inj.injection != '']
df_inj
        

,id,injection
0,1495,9741
1,1495,96143
2,1495,75523
3,1497,/51120
4,1497,4347
...,...,...
2503,292101,/9853
2504,292101,/52403
2505,292101,/123881
2506,292101,/123885


In [156]:
def test_fct(x):
    if x[0] == '/':
        return x[1:]
    return x

def to_int(x):
    try:
        return int(x)
    except:
        print (x)
        return 0

In [157]:

df_inj["injection"] = df_inj["injection"].apply(test_fct)
df_inj["injection"] = df_inj["injection"].apply(to_int)
df_inj

https://assets.serlo.org/legacy/541ae8ffd2044_a6939af75e1532d9027d2bc2e7a8d6721f73beeb.jpg
https://assets.serlo.org/legacy/5502b727644d5_a6939af75e1532d9027d2bc2e7a8d6721f73beeb.jpg
\43487
v=h116QoLFK0o


,id,injection
0,1495,9741
1,1495,96143
2,1495,75523
3,1497,51120
4,1497,4347
...,...,...
2503,292101,9853
2504,292101,52403
2505,292101,123881
2506,292101,123885


In [158]:
inj_ex = []
for ex in grouped_exercises.id:
    for row in df_inj.injection:
        if row == ex:
            inj_ex.append(row)
    

In [159]:
final_df = df_inj[df_inj.injection.isin(inj_ex)]
final_df

,id,injection
5,1499,3697
6,1499,3869
7,1499,3701
20,1509,4247
21,1509,4251
...,...,...
2502,292101,9837
2503,292101,9853
2504,292101,52403
2505,292101,123881


In [148]:
final_df.to_csv("grouped_exercise_injections", index=False)